In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import os
import numpy as np
import yaml
from dataset.ConcatBaselineDataset import ConcatBaselineDataset
from baseline.models.ConcatBaselineNet import ConcatBaselineNet
from tensorboardX import SummaryWriter

In [2]:
with open('baseline.yaml' ,'r') as f:
    config = yaml.load(f)
config = config['baseline_options']

/home/bat34/venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [3]:
config

{'batch_size': 16, 'dropout': 0.25, 'txt_enc': 'BayesianUniSkip'}

In [4]:
train_dataset = ConcatBaselineDataset(split="train", txt_enc=config['txt_enc'])
val_dataset = ConcatBaselineDataset(split="val", txt_enc=config['txt_enc'])

In [11]:
#TODO: Specify num_workers if training in script
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=config['batch_size'], collate_fn=train_dataset.collate_fn)
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=config['batch_size'], collate_fn=val_dataset.collate_fn)

In [24]:
input_dim = list(train_dataset[0]['concat_vector'].size())[0]
out_dim = len(train_dataset.ans_to_aid)

3000

In [70]:
def get_hidden_layer_list(input_dim, out_dim, size):
    if size <= 1:
        return [input_dim, out_dim]
    gap = input_dim - out_dim
    if gap % size == 0:
        repeat = size
    else:
        repeat = size - 1
    hidden_list = [input_dim]
    step = gap // size
    for i in range(1, repeat + 1):
        hidden_list.append(input_dim - i * step)
    if gap % size != 0:
        hidden_list.append(out_dim)
    return hidden_list

def create_summary_writer(model, loader, logdir):
    batch = next(iter(loader))
    writer = SummaryWriter(logdir=logdir)
    try:
        writer.add_graph(model, batch['concat_vector'])
    except Exception as e:
        print("Writer can't save model at {}".format(logdir))
        print(e)
    return writer

In [71]:
concatNet = ConcatBaselineNet(input_dim, out_dim, get_hidden_layer_list(input_dim, out_dim, 15), dropout=config['dropout'])

In [72]:
from tensorboardX import SummaryWriter
create_summary_writer(concatNet, train_loader, '/auto/homes/bat34/VQA_PartII/baseline/logs/')

In [73]:
%load_ext tensorboard
%tensorboard --logdir "/auto/homes/bat34/VQA_PartII/baseline/logs" --host "0.0.0.0" --port 6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 19321), started 0:06:18 ago. (Use '!kill 19321' to kill it.)